## Numerical SIC necromancy

Here we will try to reanimate a SIC from a small subset of the overlaps.

In [ ]:
[ activate "/Users/sflammia/Dropbox/Github/Zaunerdev"

In [1]:
using Convex
using SCS
using LinearAlgebra
using SparseArrays
using DelimitedFiles

┌ Info: Precompiling Convex [f65535da-76fb-5f13-bab9-19810c17039a]
└ @ Base loading.jl:1260
┌ Info: Precompiling SCS [c946c3f1-0d1f-5ce8-9dea-7daa1f7e2d13]
└ @ Base loading.jl:1260


Computes the overlap $\sqrt{d+1}\, \mathrm{Tr}(D^\dagger_p M)$. For a SIC this is a complex unit whenever $p\neq0$. 

In [2]:
function overlap(M,x,z)
    # The computes the overlap of a matrix M with the (x,z) Weyl-Heisenberg operator in dimension d.
    # If the input is a vector v, it efficiently computes the overlap for the matrix v*v'.
    d = size(M)[1]
    τ = -exp(pi*im/d)
    ω = exp(2*pi*im/d)
    s = 0
    if length(size(M))==2
        for j=0:d-1
            s += ω^(-j*z) * M[j+1,mod(j-x,d)+1]
        end
    elseif length(size(M))==1
        for j=0:d-1
            s += ω^(-j*z) * M[j+1]*conj(M[mod(j-x,d)+1])
        end
    end
    
    return sqrt(d+1)*τ^(x*z)*s
end

overlap (generic function with 1 method)

Return a fiducial vector from the Scott numerical paper arXiv:1703.03993.

In [3]:
function fiducial(d,fid)
    psi = readdlm("$(pwd())/Scott_solutions/sicfiducial.$(d).$(fid).150.txt")
    psi = vec([parse(ComplexF64,x*"m") for x in psi])
    return psi
end

fiducial (generic function with 1 method)

This function is a convex optimization that attempts to find a low-rank matrix consistent with a list of linear functions of the matrix and a positivity constraint. The precision and max iterations are hard-coded at the moment.

In [14]:
function necromancy(d,olps)
    M = HermitianSemidefinite(d,d)
    obj = real(tr(M))
    cons = real(tr(M)) >= 0
    for j=1:size(olps)[1]
        cons += overlap(M,Int(olps[j,1]),Int(olps[j,2])) == olps[j,3]
    end
    prob = minimize(obj,cons)
    solve!(prob, SCS.Optimizer(verbose=0,eps=1.0e-9,max_iters=1e5))
    return evaluate(M)
end

necromancy (generic function with 1 method)

In [15]:
# Pick a subset of the indices at random by Bernoulli sampling with some probability
function sampleoverlaps(p,psi)
    d = length(psi)
    inds = findall(x->x<p, rand(d,d))
    olps = zeros(Complex{Float64},length(inds),3)
    for j=1:length(inds)
        olps[j,1:3] = [inds[j][1], inds[j][2], overlap(psi,inds[j][1],inds[j][2])]
    end
    
    return olps
end

sampleoverlaps (generic function with 1 method)

Now fix a fiducial, sample some overlaps at random, and perform "matrix necromancy". Note: on Steve's laptop machine, $d=147$ can be done in under 5 minutes. $d=199$ took a little over 13 minutes on Steve's desktop. 

In [6]:
d = 51;  fid = "a"
psi = fiducial(d,fid)

olps = sampleoverlaps(3/d,psi)

@time M = necromancy(d,olps)

# M is an approxiamte SIC if this vector is numerically zero
[norm(M*M-M),norm(M-M'),real(tr(M))-1,norm(M-psi*psi')]

 34.794279 seconds (77.93 M allocations: 4.745 GiB, 5.92% gc time)


4-element Array{Float64,1}:
 6.37891695697154e-11
 2.586763933423377e-11
 5.68929348077063e-11
 1.048126745777348e-10

Now we are ready for prime time. We can try to actually perform numerical necromancy with some overlaps that Marcus has calculated using Gene's method.

In [7]:
function readoverlaps(d,fid)
    psi = readdlm("$(pwd())/d100_overlaps/overlaps_d100_fid$(fid).csv",',')
    # psi = readdlm("$(pwd())/Appleby_overlaps/overlaps_$(d)$(fid).csv",',')
    # psi = readdlm("$(pwd())/dim50_from_orbits_4_and_5/overlaps_d$(d)_number$(fid).csv",',')
    olps = vec([parse(ComplexF64,x[1:end-2]*"im") for x in psi[:,3]])
    pvals = ComplexF64.(psi[:,1:2])
    psi = hcat([pvals olps])
    return psi
end

readoverlaps (generic function with 1 method)

Lastly, we need a verification step to actually check that this produces a SIC. We check that the first column has the correct overlaps.

In [8]:
function sictest(v)
    # returns a numerical check that all of the overlaps have the correct absolute value
    d = length(v)
    siccheck = abs.([ overlap(v,x,z) for x in 1:d for z in 1:d ])/(v'*v)
    siccheck[d^2] += -sqrt(d+1)+1
    return maximum(abs.(siccheck.-1))
end

sictest (generic function with 1 method)

In [11]:
d = 100;

fid=5;

olps = readoverlaps(d,fid)


7200×3 Array{Complex{Float64},2}:
 150.0+0.0im   99.0+0.0im   0.588778-0.808295im
  50.0+0.0im   97.0+0.0im  -0.673051+0.739596im
  50.0+0.0im   93.0+0.0im  -0.960848+0.277078im
 150.0+0.0im   91.0+0.0im  -0.974086+0.226179im
  50.0+0.0im   89.0+0.0im   0.918751-0.394838im
 150.0+0.0im   87.0+0.0im  0.0936928-0.995601im
 150.0+0.0im   83.0+0.0im  -0.242092-0.970253im
  50.0+0.0im   81.0+0.0im  -0.971607-0.236599im
 150.0+0.0im   79.0+0.0im  -0.690334+0.72349im
  50.0+0.0im   77.0+0.0im   0.266141+0.963934im
  50.0+0.0im   73.0+0.0im   0.505574-0.862783im
 150.0+0.0im   71.0+0.0im   0.280374-0.959891im
  50.0+0.0im   69.0+0.0im  -0.997461+0.0712216im
      ⋮                    
 151.0+0.0im   62.0+0.0im   0.325577+0.945516im
 101.0+0.0im  161.0+0.0im   0.989472-0.144727im
  51.0+0.0im   60.0+0.0im  -0.758278-0.651932im
   1.0+0.0im  159.0+0.0im  -0.999367-0.0355869im
 151.0+0.0im   58.0+0.0im   0.579099-0.815257im
 101.0+0.0im  157.0+0.0im     0.9834-0.181451im
  51.0+0.0im   56.0+0.0im

So many overlaps! We maybe need only a few hundred... maybe 400 random ones?

In [12]:
using Random
rng = MersenneTwister(1234);
sinds = randperm(length(olps[:,1]))[1:4*d];
solps = olps[sinds,:]

400×3 Array{Complex{Float64},2}:
  89.0+0.0im   10.0+0.0im   -0.671224+0.741255im
  21.0+0.0im  176.0+0.0im    0.916317+0.400453im
  83.0+0.0im  113.0+0.0im   -0.483624-0.875276im
 103.0+0.0im   71.0+0.0im   -0.641302-0.767289im
 106.0+0.0im   51.0+0.0im    0.997792+0.0664138im
 117.0+0.0im  193.0+0.0im    0.924435-0.381339im
  57.0+0.0im  158.0+0.0im    0.172049-0.985088im
 160.0+0.0im  191.0+0.0im    0.965013-0.262202im
  53.0+0.0im  151.0+0.0im   -0.960255+0.279123im
 114.0+0.0im   51.0+0.0im    0.807242+0.590221im
  57.0+0.0im   23.0+0.0im    0.971296+0.237873im
 185.0+0.0im  156.0+0.0im   -0.307573-0.951525im
  74.0+0.0im   59.0+0.0im   -0.934446+0.356105im
      ⋮                    
  14.0+0.0im   89.0+0.0im    0.446098-0.894984im
  75.0+0.0im   52.0+0.0im  -0.0979432-0.995192im
  55.0+0.0im  124.0+0.0im   -0.560489+0.828162im
 141.0+0.0im   50.0+0.0im   -0.974086+0.226179im
  75.0+0.0im  148.0+0.0im    0.131128+0.991365im
  87.0+0.0im   65.0+0.0im     0.45361+0.8912im
  64.0+0.

In [17]:
d = 100;

fids = [7,8]
# w=zeros(length(fids),d)*(1+im)

for fid=fids
    olps = readoverlaps(d,fid)
    
    # choose a random set of 4*d overlaps from the list
    sinds = randperm(length(olps[:,1]))[1:4*d];
    solps = olps[sinds,:]
    
    print("Fid: ",fid,"\n")
    @time M = necromancy(d,solps)
    
    # M is an approxiamte SIC if this vector is numerically zero
    t1 = [norm(M*M-M),norm(M-M'),real(tr(M))-1]
    
    # If t1 is small, then this line validates that M = v*v' is a WH SIC fiducial.
    nv = findmax([ norm(M[x,:]) for x in 1:d])[2]
    v = M[nv,:]
    t2 = sictest(v)
    
    writedlm("$(pwd())/d100_overlaps/fiducial_d100_fid$(fid).csv", v)
    # w[fid,:] = v
    
    print("\nConvergence tests: ",t1,"\nSIC test: ",t2,"\n\n")
end

Fid: 7
123.967797 seconds (5.73 M allocations: 15.664 GiB, 5.27% gc time)

Convergence tests: [3.046753632300795e-11, 1.992018581248255e-11, -7.296430126757514e-11]
SIC test: 2.4125201836255883e-10

Fid: 8
 88.176129 seconds (5.70 M allocations: 15.543 GiB, 5.40% gc time)

Convergence tests: [4.404347703355039e-11, 2.0437962721677053e-11, -7.913525390534915e-11]
SIC test: 1.9573742626732837e-10



In [91]:
Diagonal(exp.(-im*atan.(imag.(w[:,1])./real.(w[:,1]))))*w

4×100 Array{Complex{Float64},2}:
  0.025346+1.73472e-18im  …  -0.0178446+0.00340733im
 0.0366846+0.0im              0.0275028-0.00924401im
 0.0103479-6.50521e-19im     9.11484e-6-0.00456032im
 0.0212559-1.73472e-18im     -0.0125596-0.0123763im

In [ ]:
# d = 50;

# Things that seem to be SICs: fid = 1, 17, 33, 54, 70, 86

for fid=fids
    olps = readoverlaps(d,fid)
    
    print("Fid: ",fid,"\n")
    @time M = necromancy(d,olps)
    
    # M is an approxiamte SIC if this vector is numerically zero
    t1 = [norm(M*M-M),norm(M-M'),real(tr(M))-1]
    
    # If t1 is small, then this line validates that M = v*v' is a WH SIC fiducial.
    nv = findmax([ norm(M[x,:]) for x in 1:d])[2]
    v = M[nv,:]
    t2 = sictest(v)
    
    print("\nConvergence tests: ",t1,"\nSIC test: ",t2,"\n\n")
end

In [9]:
fids = [1, 17, 33, 54, 70, 86]

6-element Array{Int64,1}:
  1
 17
 33
 54
 70
 86

In [10]:
d = 50;

# Things that seem to be SICs: fid = 1, 17, 33, 54, 70, 86

for fid=fids
    olps = readoverlaps(d,fid)
    
    print("Fid: ",fid,"\n")
    @time M = necromancy(d,olps)
    
    # M is an approxiamte SIC if this vector is numerically zero
    t1 = [norm(M*M-M),norm(M-M'),real(tr(M))-1]
    
    # If t1 is small, then this line validates that M = v*v' is a WH SIC fiducial.
    nv = findmax([ norm(M[x,:]) for x in 1:d])[2]
    v = M[nv,:]
    t2 = sictest(v)
    
    print("\nConvergence tests: ",t1,"\nSIC test: ",t2,"\n\n")
end

Fid: 1
  6.789073 seconds (4.22 M allocations: 3.064 GiB, 14.52% gc time)

Convergence tests: [2.5664282334684727e-6, 7.623079418006929e-7, -2.492950777477354e-6]
SIC test: 5.201273127641848e-6

Fid: 17
  6.227351 seconds (4.22 M allocations: 3.064 GiB, 21.06% gc time)

Convergence tests: [2.1379809620600693e-6, 7.106784311759002e-7, 1.9644835338450406e-6]
SIC test: 6.109531318188033e-6

Fid: 33
  5.938534 seconds (4.22 M allocations: 3.064 GiB, 11.09% gc time)

Convergence tests: [8.213671525018414e-7, 4.300452106906564e-7, -1.0104797081300632e-6]
SIC test: 2.958893209781266e-6

Fid: 54
  4.806342 seconds (4.22 M allocations: 3.064 GiB, 19.72% gc time)

Convergence tests: [3.2594374778669763e-6, 5.476078219975302e-6, -2.2909552221950236e-6]
SIC test: 8.401962689807974e-6

Fid: 70
  5.192159 seconds (4.22 M allocations: 3.064 GiB, 18.62% gc time)

Convergence tests: [1.577270888797703e-6, 8.343628000084659e-7, 4.230866631926844e-6]
SIC test: 3.932731327482841e-6

Fid: 86
  4.258168 sec

This is the test for a single fiducial.

In [11]:
d = 14;  fid = "B";

d = 50;  fid = 1;

olps = readoverlaps(d,fid)

@time M = necromancy(d,olps)

# M is an approxiamte SIC if this vector is numerically zero
[norm(M*M-M),norm(M-M'),real(tr(M))-1]

  5.718236 seconds (4.22 M allocations: 3.064 GiB, 25.60% gc time)


3-element Array{Float64,1}:
  2.5664282334684727e-6
  7.623079418006929e-7
 -2.492950777477354e-6

In [16]:
# Which eigenvalues are larger than 1e-8?
eigs = real.(eigvals(M))
eigs[eigs .> 1e-8]

1-element Array{Float64,1}:
 1.0000000009057561

To make the test as stable as possible, pick the row that has the largest norm. 

In [17]:
nv = findmax([ norm(M[x,:]) for x in 1:d])[2]
v = M[nv,:]
@time sictest(v)

  0.199584 seconds (931.20 k allocations: 47.128 MiB)


2.873524085345025e-9

In [11]:
eigs=eigvals(M)

40-element Array{Complex{Float64},1}:
 -4.4372829997174044e-11 - 7.1966393862209404e-12im
  -4.258890847202729e-11 + 5.50728038642841e-12im  
  -3.986726827405734e-11 + 9.572006466797433e-12im 
 -3.5517481299321066e-11 - 1.3711461897190236e-11im
 -3.1423160015487227e-11 - 6.844268575996989e-12im 
 -2.6546723109753044e-11 + 5.899467927593808e-12im 
 -2.6411139083305816e-11 + 1.538431233624157e-11im 
 -2.5384158067171867e-11 + 2.5271459778702706e-12im
 -2.4503935259293114e-11 + 2.284628238304129e-11im 
  -2.110339510906203e-11 - 5.110645781978718e-12im 
 -1.8284290868523563e-11 - 1.971488758856695e-11im 
 -1.6346663576616787e-11 + 2.1482174897273223e-11im
  -1.541010836082317e-11 + 1.0500994889193343e-12im
                         ⋮                         
  1.0151235015922697e-11 + 7.326395959059499e-13im 
  1.5164944748498485e-11 - 2.3907120789289607e-11im
  2.0807943366319867e-11 - 9.671147719954111e-12im 
  2.9880410316636966e-11 - 3.673944819331072e-12im 
   3.076668614651613e-11 -

In [12]:
sparsify!(x, eps) = x[abs.(x) .< eps] .= zero(eltype(x));

In [13]:
sparsify!(eigs,1e-10);
real(eigs)[end-3:end]

4-element Array{Float64,1}:
 0.13286951911040443
 0.2880794672323307 
 0.2890437613984702 
 0.2900072523009104 

## Another approach using permutation matrices (dumb)

Let us consider now the set of *all* the SIC phase overlaps, $\mathbf{b} = \{\mathrm{e}^{i \theta_{\mathbf{p}}} : \mathbf{p}\in\mathbb{Z}_d^2\}$. Let $\mathcal{A}$ be the map $\mathcal{A} : \mathrm{Mat}_{d\times d}(\mathbb{C}) \to \mathbb{C}^{d^2}$ given by 
$$\mathcal{A}(M)_j = \sqrt{d+1}\, \mathrm{Tr}(D^\dagger_{\mathbf{p}} M)$$
where $D_{\mathbf{p}}$ is a displacement operator.
Now consider the following convex optimization
\begin{align}
    \max_{M,S}\quad & \mathrm{Tr}(M) \\
    \mathrm{s.t.}\quad & M\succeq 0\\
    & \mathcal{A}(M) = S \mathbf{b}\\
    & S \ge 0\\
    & S \mathbf{1} = \mathbf{1}\\
    & S^T \mathbf{1} = \mathbf{1}
\end{align}
Here the constraints specify that $S$ is a doubly stochastic operator.

In [89]:
d = 4;  fid = "a"
psi = fiducial(d,fid)

olps = [ overlap(psi,x,z) for x in 1:d for z in 1:d ]

M = HermitianSemidefinite(d,d)
#S = Variable(d^2,d^2,Positive())
#cons = S*ones(d^2) == ones(d^2)
#cons += S'*ones(d^2) == ones(d^2)

S = ComplexVariable(d^2,d^2)
cons = real(S)>=0
cons += imag(S)==0
cons += S*ones(d^2) == ones(d^2)
cons += S'*ones(d^2) == ones(d^2)


cons

#@time M = necromancy(d,olps)

# M is an approxiamte SIC if this vector is numerically zero
#[norm(M*M-M),norm(M-M'),real(tr(M))-1,norm(M-psi*psi')]

4-element Array{Constraint,1}:
 >= constraint (affine)
├─ real (affine; real)
│  └─ 16×16 complex variable (id: 790…269)
└─ 0                                                                                               
 == constraint (affine)
├─ imag (affine; real)
│  └─ 16×16 complex variable (id: 790…269)
└─ 0                                                                                               
 == constraint (affine)
├─ * (affine; complex)
│  ├─ 16×16 complex variable (id: 790…269)
│  └─ 16-element Array{Float64,1}
└─ 16-element Array{Float64,1}                                   
 == constraint (affine)
├─ * (affine; complex)
│  ├─ adjoint (affine; complex)
│  │  └─ 16×16 complex variable (id: 790…269)
│  └─ 16-element Array{Float64,1}
└─ 16-element Array{Float64,1}

In [102]:
S*olps
[ overlap(M,x,z) for x in 1:d for z in 1:d ]


16-element Array{Convex.MultiplyAtom,1}:
 * (affine; complex)
├─ -1.58114-1.58114im
└─ + (affine; complex)
   ├─ 0
   ├─ * (affine; complex)
   │  ├─ 1.0+0.0im
   │  └─ index (affine; complex)
   │     └─ …
   ├─ * (affine; complex)
   │  ├─ 6.12323e-17-1.0im
   │  └─ index (affine; complex)
   │     └─ …
   ├─ * (affine; complex)
   │  ├─ -1.0-1.22465e-16im
   │  └─ index (affine; complex)
   │     └─ …
   └─ * (affine; complex)
      ├─ -1.83697e-16+1.0im
      └─ index (affine; complex)
         └─ …    
 * (affine; complex)
├─ 4.96507e-16+2.23607im
└─ + (affine; complex)
   ├─ 0
   ├─ * (affine; complex)
   │  ├─ 1.0+0.0im
   │  └─ index (affine; complex)
   │     └─ …
   ├─ * (affine; complex)
   │  ├─ -1.0-1.22465e-16im
   │  └─ index (affine; complex)
   │     └─ …
   ├─ * (affine; complex)
   │  ├─ 1.0+2.44929e-16im
   │  └─ index (affine; complex)
   │     └─ …
   └─ * (affine; complex)
      ├─ -1.0-3.67394e-16im
      └─ index (affine; complex)
         └─ … 
 * (affine; com

In [52]:
function reanimation(d,olps)
    M = HermitianSemidefinite(d,d)
    S = Variable(d^2,d^2, Positive())
    cons = S*ones(d^2) == ones(d^2)
    cons += S'*ones(d^2) == ones(d^2)
    cons += Alaps(M) == S*olps
    obj = real(tr(M))
    prob = minimize(obj,cons)
    solve!(prob, SCSSolver(verbose=1,eps=1.0e-8,max_iters=1e4))
    return evaluate(M)
end

reanimation (generic function with 1 method)

In [53]:
reanimation(4,olps)

MethodError: MethodError: no method matching ispos(::Array{Convex.MultiplyAtom,1})
Closest candidates are:
  ispos(!Matched::Real) at /Users/sflammia/.julia/packages/Convex/NBqGs/src/constant.jl:8
  ispos(!Matched::Union{AbstractArray{#s27,1}, AbstractArray{#s27,2}} where #s27<:Real) at /Users/sflammia/.julia/packages/Convex/NBqGs/src/constant.jl:9

In [68]:
S>=0
T=Variable(3,3,Positive())

Variable
size: (3, 3)
sign: positive
vexity: affine
id: 126…249

In [66]:
imag(T)==0
real(T)>=0

>= constraint (affine)
├─ real (affine; real)
│  └─ 3×3 complex variable (id: 131…774)
└─ 0

In [69]:
T*ones(3)

* (affine; positive)
├─ 3×3 positive variable (id: 126…249)
└─ [1.0, 1.0, 1.0]

In [97]:
τ = -exp(pi*im/d)

sqrt(d+1)*τ^(x*z)*sum(exp(2*pi*im*(-j*z)/d) * M[j+1,mod(j-x,dd)+1] for dd=0:d-1)

[ sqrt(d+1)*(-exp(pi*im*x*z/d)) * sum(exp(2*pi*im*(-j*z)/d) * M[j+1,mod(j-x,d)+1] for j=0:d-1) for x in 1:d for z in 1:d ]

UndefVarError: UndefVarError: x not defined

# Temp

The following $\delta$-notation for the "half-theta product" or q-Pochhammer symbol is introduced in Kopp2020d.

Define the function
$$
\delta(z;\tau) = E_{e(\tau)}(e(z)) = \prod_{j=0}^\infty \left(1-e(z+j\tau)\right).
$$

The "signed" conjecture may be rephrased as follows, using the alternative expression for the class invariants introduced in Kopp2020d.

Let $d>3$ be a positive integer, and let $\Delta = (d+1)(d-3)$ and $\varepsilon = \frac{d-1+\sqrt{\Delta}}{2}$.
Let $Q$ be a (not necessarily primative) binary quadratic form of discriminant $\Delta$.
Let $\omega_Q^- < \omega_Q^+$ be the roots of $Q(\omega,1)=0$, and let $\ell = \ell_+ - \ell_-$ be the difference between the lengths of the Hirzebruch-Jung continued fraction expansions of $\omega_Q^+$ and $\omega_Q^-$, respectively.  
Let $G$ be a generator for the stabaliser of $Q$ under the action of $\Gamma(d)$ so that $\lim_{t \to \pm \infty} Q^t\tau = \omega_Q^{\pm}$ respectively for all $\tau \in \mathcal{H}$.
For $\tau \in \mathcal{H}$, define the function
$$
\kappa_{m,n}(t) = -\zeta_8^{\ell} \exp\left(\frac{(d^2-1)\pi i\mathrm{Norm}(m+n\omega)}{d}\right) \frac{\delta\left(\frac{m+nG^{t+1}\tau}{d}, G\tau\right)}{\delta\left(\frac{m+n\tau}{d}, \tau\right)}.
$$
Now, for $0 \leq m,n < d$, set
$$
\nu_{m,n} = \left\{\begin{array}{ll}
1, &\mbox{ if } m=n=0; \\
\lim_{\tau \to \omega_Q^+} \frac{1}{\sqrt{d+1}}\kappa_{m,n}(\tau), &\mbox{ otherwise}.
\end{array}\right.
$$
Then, the $\nu_{m,n}$ are ghost SIC overlaps in dimension $d$. In particular, the $\nu_{m,n}$ are real, and the matrix
$$
M = \frac{1}{d}\sum_{m,n} \nu_{m,n} D_{m,n}^\dagger
$$
is a rank 1 idempotent.

In [25]:
d=7
Δ=(d+1)*(d-3)
𝜖=(d-1+sqrt(Δ))/2
a=1
b=mod(Δ,2)
c=Int((mod(Δ,2)-Δ)/4)
Q=[[a,b/2],[b/2,c]]
disc=b^2-4*a*c

32

In [26]:
Q

2-element Array{Array{Float64,1},1}:
 [1.0, 0.0]
 [0.0, -8.0]

In [20]:
Q*[w,1]

UndefVarError: UndefVarError: w not defined

In [37]:
1/(1/(1/(1/(1/(sqrt(8)-2)-1)-4)-1)-4)-1

0.20710678118620618

In [51]:
1/(1/(1/(1/(1/(1/(-sqrt(8)+3)-5)-1)-4)-1)-4)

1.207106781174952

In [ ]:
sqrt(8)
meyerl=0